In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

import json
import os
import shutil
from distutils.dir_util import copy_tree
import random
import re
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models


%matplotlib inline

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(3001)

Copy files to the work directory so we can split them to train and validation sets:

In [ ]:
def move_files(source_folder, destination_folder, files_to_move):
    # iterate files
    for file in files_to_move:
        # construct full file path
        source = source_folder + file
        destination = destination_folder + file
        # move file
        shutil.move(source, destination)

In [ ]:
import PIL
import random

data_dir = '../input/create-rotated-hotel-images-dataset/data'
    
train_files = [f for f in os.listdir(os.path.join(data_dir, 'images/train/class/'))]#'./data/images/train/class/')]
valid_files = [f for f in os.listdir(os.path.join(data_dir, 'images/valid/class/'))]#./data/images/valid/class/')]

Load labels data as dictionary. The keys are the file names and the values are the labels:

In [ ]:
with open(os.path.join(data_dir, 'label.json'), 'r') as f:
    labels_list = json.load(f)
    
labels_dict = {''.join([str(key), '.png']): value for key, value in enumerate(labels_list)}

Define the necessary transformations for the images:

In [ ]:
# required parameters for all images fed into the pretrained models:
mean_for_norm = np.array([0.485, 0.456, 0.406])
std_for_norm = np.array([0.229, 0.224, 0.225])
image_size = 224

transform = transform=transforms.Compose([transforms.Resize(image_size),
                                          transforms.RandomResizedCrop(image_size),
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean_for_norm, std_for_norm),
                                          transforms.RandomErasing()])

Modify ImageFolder method to apply to regression tasks:

In [ ]:
class RegressionImageFolder(torchvision.datasets.ImageFolder):
    def __init__(self, root, image_scores, **kwargs) -> None:
        super().__init__(root, **kwargs)
        paths, _ = zip(*self.imgs)
        file_names = [re.search(r'[\d]+.png',path).group() for path in paths]
        self.targets = [image_scores[file_name] for file_name in file_names]
        self.samples = self.imgs = list(zip(paths, self.targets))

Create the datasets:

In [ ]:
trainset = RegressionImageFolder(os.path.join(data_dir, 'images/train/'), labels_dict, transform=transform)
validset = RegressionImageFolder(os.path.join(data_dir, 'images/valid/'), labels_dict, transform=transform)

Load the datases:

In [ ]:
batch_size = 32

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
validloader = DataLoader(validset, batch_size=batch_size, shuffle=True, num_workers=1)

Inspect the first few images and labels:

In [ ]:
samples, targets = next(iter(trainloader))

grid = torchvision.utils.make_grid(samples, nrow=8) 
grid = grid.permute(1,2,0) * std_for_norm + mean_for_norm

plt.figure(figsize=(15,15))
plt.imshow(grid)
plt.axis('off')

print('targets:\n', targets.reshape(-1,8).numpy())

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Load pretrained VGG16 model for transfer learning:

In [ ]:
model = models.vgg16(pretrained=True)
print(model)

Freeze all feature extraction layers:

In [ ]:
for param in model.features.parameters():
    param.requires_grad = False

Replace the output layer with a single node:

In [ ]:
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 1)

Train the model:

In [ ]:
def save_checkpoint(model, optimizer, epoch, avg_loss, train_rmse, val_rmse):
    torch.save({"model": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "epoch": epoch,
                "avg_loss": avg_loss,
                "train_rmse": train_rmse,
                "val_rmse": val_rmse
               }, "model.pt")
print("Saved the checkpoint")

In [ ]:
# move model to gpu if avaliable:
model.to(device=device, dtype = torch.float32)

# set hyperparameters:
epochs = 1
lr = 0.00001#0.0001#0.001

# define loss and optimizer:
loss_criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr) 

train_rmse_log = []
val_rmse_log = []
patience = 0

if os.path.isdir('../input/imagerotationmodelcheckpoint'):
    checkpoint = torch.load('../input/imagerotationmodelcheckpoint/model.pt', map_location=device)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    model.eval()
else:
    for epoch in range(epochs):

        # training:
        model.train()
        total_loss = 0
        for images, labels in tqdm(trainloader, desc=f'Epoch {epoch} Training'):

            images = images.to(device=device, dtype = torch.float32)
            labels = labels.to(device=device, dtype = torch.float32)

            preds = model(images).squeeze()
            loss = loss_criterion(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * batch_size

        avg_loss = total_loss/len(trainset)

        train_rmse = np.sqrt(avg_loss)
        train_rmse_log.append(train_rmse)


        # evaluation:
        model.eval()
        with torch.no_grad():

            valid_loss = 0
            for images_valid, labels_valid in tqdm(validloader, desc=f'Epoch {epoch} Evaluation'):

                images_valid = images_valid.to(device=device, dtype = torch.float32)
                labels_valid = labels_valid.to(device=device, dtype = torch.float32)

                pred_valid = model(images_valid).squeeze()
                valid_loss += loss_criterion(pred_valid, labels_valid).item() * batch_size

            avg_valid_loss = valid_loss/len(validset)

            val_rmse = np.sqrt(avg_valid_loss)      
            val_rmse_log.append(val_rmse)

        print(f"Epoch: {epoch} \ntrain RMSE: {train_rmse:2.4}, validation RMSE: {val_rmse:2.4}")

        save_checkpoint(model, optimizer, epoch, avg_loss, train_rmse, val_rmse)
        
        # Break if overfitting occures
        if (val_rmse - train_rmse) >= 1.0:
            patience += 1
            if patience > 2:
                print("Overfitting has been detected - trainig will now stop")
                break
        else:
            patience = 0

        # break if the rmse plateaus:
        plateau_length = 15
        if len(train_rmse_log) >= plateau_length:
            range_last = max(train_rmse_log[-plateau_length:]) - min(train_rmse_log[-plateau_length:])
            if range_last <= 0.5:
                print("The model has reached a plateau - trainig will now stop")
                break

In [ ]:
from IPython.display import FileLink
FileLink('./model.pt')